In [1]:
import openai 
import random
import pandas as pd

# Determine the model: can be gpt-3.5-turbo or gpt-4
model_name = "gpt-3.5-turbo"

In [2]:
try:
  api_key = "INSERT KEY" # Add API key
  openai.api_key = api_key
  print("API key successfully loaded.")
except:
  print("No API key file found.")

API key successfully loaded.


In [3]:
# CSV path (change accordingly)
csv_path = "test_TF.csv" 
prompts = []
fields = ['Instructions', 'Condition', 'Study', 'Proposition', 'NAND', 'NOR']  # Columns used in CSV

# Reading CSV to dataframe
prompts_df = pd.read_csv(csv_path, usecols=fields)

for _, row in prompts_df.iterrows():
    prompt = {}

    # Check if either "A" or "B" exists in the row
    choices = {"NAND":row["NAND"], "NOR":row["NOR"]}
    
    text_parts = [row['Instructions'], row['Proposition'], "A:", row["NAND"], "B:",row["NOR"]]
    
    # Filter out empty strings before joining
    prompt["text"] = ' '.join(filter(None, text_parts))
    prompt["condition"] = row['Condition']
    prompt["study"] = row['Study']
    
    prompts.append(prompt)

random.shuffle(prompts)

print(f"Found {len(prompts)} prompts in the given CSV file.")

Found 5 prompts in the given CSV file.
Given the following sentence, option A is the best continuation. Is the previous sentence true or false? John is upset. Clearly, he didn’t pass the semantics or the syntax exam. A: I wonder which it was. B: And it wouldn’t be the first time he failed those classes.
Given the following sentence, option A is the best continuation. Is the previous sentence true or false? If I remember correctly, Mary didn’t invite John or Suzi to her birthday party. A: I don’t know which of them. B: She’s upset with both of them and doesn’t want to see them.
Given the following sentence, option A is the best continuation. Is the previous sentence true or false? It’s cold in this house. I bet you, John didn’t close the window or the back door. A: I wonder which one of those two we’ll find open. B: Look, just as I said, both of them are wide open!
Given the following sentence, option A is the best continuation. Is the previous sentence true or false? If I remember corr

In [4]:
outputs = []
total_tokens = 0

# Calling chatGPT and appending responses to outputs list
for prompt in prompts:
  response = openai.ChatCompletion.create(
                model=model_name,
                messages=[
                      {"role": "system", "content": "If prompted with a true or false question, only respond with either 'true' or 'false' in lowercase."},
                      {"role": "user", "content": prompt["text"]},
                  ]
              )
  gpt_answer = response['choices'][0]['message']['content']
  outputs.append(gpt_answer)

  total_tokens += response["usage"]["total_tokens"]

print(f"Messages generated with {total_tokens} tokens")

Messages generated with 481 tokens


In [5]:
df = pd.DataFrame({"Prompt": [prompt["text"] for prompt in prompts],  
                   "Condition": [prompt["condition"] for prompt in prompts],
                   "Study": [prompt["study"] for prompt in prompts],
                   "Response": outputs})

# Save the DataFrame to a CSV file
output_csv_path = "output_responses_TF.csv"
df.to_csv(output_csv_path, index=False)

print(f"Responses saved to {output_csv_path}")

Responses saved to output_responses_TF.csv
